# Relatório 2

**Nome:** Thiago Lopes <br>
**Matrícula:** 20100358 <br>
**Turma:** T2

# Bibliotecas python utilizadas

In [110]:
import math
import matplotlib.pyplot as plt
import numpy as np
from prettytable import PrettyTable
import sympy


# Funçoes Auxiliares

In [111]:
def pivot(A, b, k):
    max_index = np.argmax(np.abs(A[k:, k])) + k
    
    if max_index != k:
        A[[k, max_index]] = A[[max_index, k]]
        b[[k, max_index]] = b[[max_index, k]]


In [112]:
def back_substitution(U, y):
    m = len(y)
    x = np.zeros(m)
    
    for i in range(m - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]
        
    return x


In [113]:
def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros_like(b)
    
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
    
    return y


In [114]:
def forward_substitution_LU(L, b, Pivot):
    m = len(b)
    y = np.zeros(m)
    
    for i in range(m):
        y[i] = b[Pivot[i] - 1] - np.dot(L[i, :i], y[:i])        
    return y


# Métodos implementados: Gaussian Elimination, LU Decomposition, Cholesky Decomposition,:

In [115]:
def gauss_elimination(A, b, use_pivoting=False):
    n = len(b)
    A = A.astype(float)
    b = b.astype(float)

    for k in range(n-1):
        if use_pivoting:  
            pivot(A, b, k)

        for i in range(k+1, n):
            m = A[i][k] / A[k][k]
            A[i][k] = 0
            
            for j in range(k+1, n):
                A[i][j] -= m * A[k][j]
                
            b[i] -= m * b[k]

    return back_substitution(A, b)


In [116]:
def LU_decomposition(A, b, use_pivoting=False):
    A = A.astype(np.float64)
    b = b.astype(np.float64)

    m, n = A.shape
    
    if m != n:
        raise ValueError("Matrix must be square")
    
    Pivot = np.arange(1, m + 1)
    PdU = 1.0
    Info = 0
    LU = A.copy()

    for j in range(n):
        if use_pivoting:
            pivot(LU, b, j)
        
        Pivot[j], Pivot[np.argmax(np.abs(LU[j:, j])) + j] = Pivot[np.argmax(np.abs(LU[j:, j])) + j], Pivot[j]

        if LU[j, j] == 0:
            if Info == 0:
                Info = j + 1
            continue

        PdU *= LU[j, j]

        if abs(LU[j, j]) != 0:
            r = 1 / LU[j, j]
            for i in range(j + 1, m):
                Mult = LU[i, j] * r
                LU[i, j] = Mult
                LU[i, j + 1:] -= Mult * LU[j, j + 1:]
        else:
            if Info == 0:
                Info = j + 1

    if Info != 0:
        raise ValueError(f"Matrix is singular at row {Info}")

    y = forward_substitution_LU(LU, b, Pivot)

    x = back_substitution(LU, y)

    return x, LU, Pivot, PdU, Info


In [117]:
#TODO usar pivotamento

def cholesky_decomposition(A, b, use_pivoting=False):
    n = A.shape[0]
    L = np.zeros_like(A)
    Det = 1.0
    Info = 0
    
    for j in range(n):
        Soma = 0.0
        for k in range(j):
            Soma += L[j, k] * L[j, k]
        
        t = A[j, j] - Soma
        if t > 0:
            L[j, j] = np.sqrt(t)
            r = 1 / L[j, j]
            Det *= t
        else:
            Info = j + 1
            raise ValueError("Matrix is not positive definite")
        
        for i in range(j + 1, n):
            Soma = 0.0
            for k in range(j):
                Soma += L[i, k] * L[j, k]
            L[i, j] = (A[i, j] - Soma) * r
    
    y = forward_substitution(L, b)
    
    x = back_substitution(L.T, y)
    
    return x, L, Det, Info


In [118]:
def gauss_jacobi(A, b, toler, iter_max):
    n = len(b)
    x = np.zeros_like(b)
    v = np.zeros_like(b)
    iter_count = 0
    
    for i in range(n):
        x[i] = b[i] / A[i, i]
    
    while True:
        iter_count += 1
        
        for i in range(n):
            sum = 0
            for j in range(n):
                if i != j:
                    sum += A[i, j] * x[j]
            v[i] = (b[i] - sum) / A[i, i]
        
        norma_num = 0
        norma_den = 0
        
        for i in range(n):
            t = abs(v[i] - x[i])
            if t > norma_num:
                norma_num = t
            if abs(v[i]) > norma_den:
                norma_den = abs(v[i])
            x[i] = v[i]
        
        norma_rel = norma_num / norma_den
        # print(f"Iteração: {iter_count}, x: {x}, NormaRel: {norma_rel}")
        
        if norma_rel <= toler or iter_count >= iter_max:
            break
    
    if norma_rel <= toler:
        info = 0
    else:
        info = 1
    
    return x, iter_count, info


In [119]:
import numpy as np

def gauss_seidel(A, b, tol, iter_max):
    n = len(b)
    x = np.zeros_like(b, dtype=np.double)
    iter_count = 0
    info = 1

    for i in range(n):
        x[i] = b[i] / A[i, i]

    for iter_count in range(1, iter_max + 1):
        # x_old = np.copy(x)
        norma_num = 0
        norma_den = 0

        for i in range(n):
            soma = 0
            
            for j in range(n):
                if i != j:
                    soma += A[i, j] * x[j]
                    
            v = x[i]
            x[i] = (b[i] - soma) / A[i, i]
            t = abs(v - x[i])
            
            if t > norma_num:
                norma_num = t
                
            if abs(x[i]) > norma_den:
                norma_den = abs(x[i])

        norma_rel = norma_num / norma_den
        # print(f"Iteração {iter_count}: x = {x}, NormaRel = {norma_rel}")

        if norma_rel <= tol or iter_count >= iter_max:
            if norma_rel <= tol:
                info = 0
            break

    return x, iter_count, info


# Exercícios

## Exercício 1

In [120]:
# # Gauss eliminatioon
# A = np.array([
#     [2, 3, -1],
#     [4, 4, -3],
#     [2, -3, 1]])

# b = np.array([5, 3, -1])

# solution = gauss_elimination(A, b, use_pivoting=True)
# print(solution)

In [121]:
# # LU Decomposition
# A = np.array([
#     [3, 2, 4],
#     [0, 0.333, 0.666],
#     [0, 0, -8]])

# b = np.array([1, 1.666, 0])

# # -------------------------------

# # A = np.array([
# #     [1, 0, 0],
# #     [0.333, 1, 0],
# #     [1.333, 1, 1]])

# # b = np.array([1, 2, 3])

# # -------------------------------

# # A = np.array([
# #     [1, 5, 5], 
# #     [6, 9, 22], 
# #     [32, 5., 5]])

# # b = np.array([1, 2, 7])

# # -------------------------------

# # A = np.array([[1, 2, 4, 3],
# #               [2, 0, 2, 1],
# #               [4, 1, 3, 2],
# #               [1, 0, 1, 1]], dtype=float)

# # b = np.array([16, 13, 27, 7], dtype=float)

# solution, LU, Pivot, PdU, Info = LU_decomposition(A, b, True)

# print("Solution:", solution)
# print("LU matrix:\n", LU)
# print("Pivot indices:", Pivot)
# print("Product of the diagonal of U:", PdU)
# print("Info:", Info) 

In [122]:
# # cholesky_decomposition

# A = np.array([[4, 2, -4],
#                [2, 10, 4],
#                [-4, 4, 9]], dtype=float)

# b = np.array([0, 6, 5], dtype=float)

# x, L, Det, Info = cholesky_decomposition(A, b, True)

# print("Solution x:", x)
# print("Lower triangular matrix L:\n", L)
# print("Determinant:", Det)
# print("Info:", Info)

In [126]:
# Gauss Jacobi

# A = np.array([[10, -1, 2, 0],
#               [-1, 11, -1, 3],
#               [2, -1, 10, -1],
#               [0, 3, -1, 8]], dtype=float)

# b = np.array([6, 25, -11, 15], dtype=float)

A = np.array([[10, 2, 1], [1, 5, 1], [2, 3, 10]], dtype=float)

b = np.array([7, -8, 6], dtype=float)

toler = 1e-10
iter_max = 100

x, iter_count, info = gauss_jacobi(A, b, toler, iter_max)

print(f"Solução: {x}")
print(f"Iterações: {iter_count}")
print(f"Info: {info}")

Solução: [ 1. -2.  1.]
Iterações: 23
Info: 0


In [124]:
# # Gauss Seidel

# A = np.array([[10, 2, 1], [1, 5, 1], [2, 3, 10]], dtype=np.double)

# b = np.array([7, -8, 6], dtype=np.double)

# tol = 1e-6
# iter_max = 100

# x, iter_count, info = gauss_seidel(A, b, tol, iter_max)

# print(f"Solução: {x}")
# print(f"Número de iterações: {iter_count}")
# print(f"Info: {info}")
